# Project Three Notebook

For your final notebook, feel free to duplicate this notebook and edit as needed. 


## Load Some Stuff

This is where we load libraires and the like so we can do what we need. If you get an error saying a module is not loaded, open a new terminal/cmd line and try running: `pip install [module name]`. 

In [36]:
import os
try:
    inputFunc = raw_input
except NameError:
    inputFunc = input

import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import numpy as np
 
import seaborn as sns
from statsmodels.formula.api import ols

from sklearn import linear_model
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from patsy import dmatrices

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

import random



# Custom functions

def evaluate(pred, labels_test):
    acc = accuracy_score(pred, labels_test)
    print ("Accuracey: %s"%acc)
    tn, fp, fn, tp = confusion_matrix(labels_test, pred).ravel()

    recall = tp / (tp + fp)
    percision = tp / (tp + fn)
    f1 = (2 / ((1/recall)+(1/percision)))

    print ("")
    print ("True Negatives: %s"%tn)
    print ("False Positives: %s"%fp)
    print ("False Negatives: %s"%fn)
    print ("True Positives: %s"%tp)
    print ("Recall: %s"%recall)
    print ("Precision: %s"%percision)
    print ("F1 Score: %s"%f1)

def plot_bound(Z_val,data,col1,col2,binary):
    # Z-val equals "Yes" value. E.g., "Y" or "1". 
    # data equals df
    # col1 and col2 defines which colums to use from data
    # Plot binary decision boundary. 
    # For this, we will assign a color to each
    # point in the mesh [x_min, m_max]x[y_min, y_max].
    
    x_min = float(data.iloc[:,[col1]].min())-float(data.iloc[:,[col1]].min())*0.10 
    x_max = float(data.iloc[:,[col1]].max()+float(data.iloc[:,[col1]].min())*0.10)
    y_min = 0.0; 
    y_max = float(training.iloc[:,[col2]].max())+float(training.iloc[:,[col2]].max())*0.10
    h_x = (x_max-x_min)/100  # step size in the mesh
    h_y = (y_max-y_min)/100  # step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h_x), np.arange(y_min, y_max, h_y))
    if binary == 1:
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])   
        Z = np.where(Z=="Y",1,0)
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.pcolormesh(xx, yy, Z)
    plt.show()

## Data Cleaning

Here we load the data we collected and get it all ready to feed to our statistical model(s). That is, we are trying to make a table with one **target** column and one or more **features**. Here I'm loading restaurant.csv from: https://catalog.data.gov/dataset/food-inspections-lives-standard Note: you can find information on the data elements at this link. 


In [37]:
# Load and peek at your data. Change the file name as needed. 
raw_data_df = pd.read_csv('restaurant.csv', parse_dates=[0]) 
raw_data_df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,90604,Taqueria Los Amigos,5300 Mission St.,San Francisco,CA,94112,NaN,NaN,NaN,NaN,90604_20170502,5/2/17 0:00,NaN,New Ownership,90604_20170502_103133,Foods not protected from contamination,Moderate Risk
1,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,-122.420455,"(37.789784, -122.420455)",NaN,1757_20170928,9/28/17 0:00,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk
2,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,-122.419066,"(37.759174, -122.419066)",1.415583e+10,4864_20161206,12/6/16 0:00,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk
3,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,NaN,NaN,NaN,NaN,79782_20160503,5/3/16 0:00,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk
4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,NaN,NaN,NaN,NaN,76437_20160329,3/29/16 0:00,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk


In [38]:
# You can explore unique entires by stating the column and using .unique() like this:
print(raw_data_df["inspection_score"].unique())
print(raw_data_df["risk_category"].unique())

[  nan   86.   84.   92.   76.   88.  100.   82.   57.   94.   68.   78.
   71.   90.   87.   83.   72.   96.   85.   79.   80.   81.   91.   73.
   98.   89.   77.   75.   63.   70.   62.   93.   66.   74.   69.   64.
   67.   55.   56.   61.   65.   60.   58.   52.   54.   59.   53.   48.]
['Moderate Risk' 'Low Risk' 'High Risk' nan]


In [39]:
# You can explore rows with a specific value like so
raw_data_df[raw_data_df["inspection_score"]== 98].head() # remove .head() to see all entires

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
114,547,LITTLE HENRY'S,3600 BALBOA St,San Francisco,CA,94121,37.775728,-122.497522,"(37.775728, -122.497522)",NaN,547_20160412,4/12/16 0:00,98.0,Routine - Unscheduled,547_20160412_103142,Unclean nonfood contact surfaces,Low Risk
139,7689,UNOCAL 76 PACIFIC ENTERPRISES,999 OCEAN Ave,San Francisco,CA,94112,37.723133,-122.452707,"(37.723133, -122.452707)",1.415559e+10,7689_20160811,8/11/16 0:00,98.0,Routine - Unscheduled,7689_20160811_103148,No thermometers or uncalibrated thermometers,Low Risk
154,134,Mint,400 McAllister St,San Francisco,CA,94102,37.780247,-122.418974,"(37.780247, -122.418974)",NaN,134_20150806,8/6/15 0:00,98.0,Routine - Unscheduled,134_20150806_103145,Improper storage of equipment utensils or linens,Low Risk
259,6863,Cheese Plus,2001 Polk St,San Francisco,CA,94109,37.795273,-122.421665,"(37.795273, -122.421665)",NaN,6863_20160328,3/28/16 0:00,98.0,Routine - Unscheduled,6863_20160328_103149,Wiping cloths not clean or properly stored or ...,Low Risk
291,7744,SAFEWAY #759,625 MONTEREY Blvd,San Francisco,CA,94127,37.731523,-122.450050,"(37.731523, -122.45005)",1.415558e+10,7744_20160809,8/9/16 0:00,98.0,Routine - Unscheduled,7744_20160809_103144,Unapproved or unmaintained equipment or utensils,Low Risk


In [40]:
# You can count the number of rows like so
print("Size of entire table: %s "%len(raw_data_df))
print("Size of entires matching filter: %s "%len(raw_data_df[raw_data_df["risk_category"]=='Low Risk']))

Size of entire table: 50567 
Size of entires matching filter: 18892 


In [44]:
# You can invert a match by using the 'not equal' evaluation.
print("Size of entires matching filter: %s "%len(raw_data_df[raw_data_df["risk_category"]!='Low Risk']))
raw_data_df[raw_data_df["risk_category"]!='Low Risk'].head()

Size of entires matching filter: 31675 


,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,90604,Taqueria Los Amigos,5300 Mission St.,San Francisco,CA,94112,NaN,NaN,NaN,NaN,90604_20170502,5/2/17 0:00,NaN,New Ownership,90604_20170502_103133,Foods not protected from contamination,Moderate Risk
1,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,-122.420455,"(37.789784, -122.420455)",NaN,1757_20170928,9/28/17 0:00,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk
3,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,NaN,NaN,NaN,NaN,79782_20160503,5/3/16 0:00,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk
4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,NaN,NaN,NaN,NaN,76437_20160329,3/29/16 0:00,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk
5,83325,Five Star Truffles,4251 A 18th St,San Francisco,CA,94114,NaN,NaN,NaN,1.415556e+10,83325_20150529,5/29/15 0:00,NaN,New Ownership,83325_20150529_103116,Inadequate food safety knowledge or lack of ce...,Moderate Risk


In [48]:
# You can make a new table from your filtered rows like so
processed_data_df = raw_data_df[raw_data_df["inspection_score"]!=86.0]
processed_data_df = processed_data_df[processed_data_df["risk_category"]!='Low Risk']
# Note how I filtered first on raw_data_df and then on processed_data_df

# Now let's remove all students
processed_data_df = processed_data_df[processed_data_df["business_state"]!='CA']

# So how many entires are there?
print("Size of entire table: %s "%len(processed_data_df))
# Let's peak at the table.
processed_data_df.head()

Size of entire table: 0 


,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category


In [49]:
# for the special case of when a calue is NaN, you can filter based on the value not being null (i.e., empty)
processed_data_df = processed_data_df[pd.notnull(processed_data_df["inspection_score"])]
processed_data_df = processed_data_df[pd.notnull(processed_data_df["risk_category"])]
print("Size of entire table: %s "%len(processed_data_df)) # in the example data, this gets rid of a few rows
processed_data_df.head()

Size of entire table: 0 


,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category


In [ ]:
# You can remove unwanted colums like so
# for a single column
processed_data_df = processed_data_df.drop('In.general..how.similar.are.you.to.other.people.you.know._2011', 1)
# for multiple columns
processed_data_df = processed_data_df.drop([
                                            'How.satisfied.are.you.with.Somerville.as.a.place.to.live.',
                                            'How.satisfied.are.you.with.your.neighborhood.'
                                           ], 1)
processed_data_df.head()

In [ ]:
# Alternativly, if you want to make a new table from a subset of columns, you can do so like this.
processed_data_df = processed_data_df[[
                                        'How.happy.do.you.feel.right.now.', 
                                        'How.satisfied.are.you.with.your.life.in.general.', 
                                        'What.is.your.annual.household.income.',
                                        'Age.'
                                     ]].copy()
processed_data_df.head()

In [ ]:
# You can rename columns like so.
processed_data_df = processed_data_df.rename(columns={
                                                        'How.happy.do.you.feel.right.now.': 'happy', 
                                                        'How.satisfied.are.you.with.your.life.in.general.': 'satisfied',
                                                        'What.is.your.annual.household.income.': 'income',
                                                        'Age.': 'age',
                                                     })
processed_data_df.head()

In [ ]:
# You can replace values in a column based on logic like so
# Note: I used the unique values found above to inform my logic.
# That is, I took the unique text lables and translated them into numbers.
# It's clear that different surveys had different buckets. So I probably 
# sould limit myself to years using the same metrics, but for our purposes
# I'm just going to run with a quick and dirty translation. 

processed_data_df.loc[processed_data_df['income'] == 'Less than $10,000', 'income'] = 5000 
processed_data_df.loc[processed_data_df['income'] == '10,000 - $19,999', 'income'] = 15000
processed_data_df.loc[processed_data_df['income'] == '$10,000 to $24,999', 'income'] = 17500
processed_data_df.loc[processed_data_df['income'] == '20,000 - $29,999', 'income'] = 25000
processed_data_df.loc[processed_data_df['income'] == '$25,000 to $49,999', 'income'] = 37500
processed_data_df.loc[processed_data_df['income'] == '30,000 - $39,999', 'income'] = 35000
processed_data_df.loc[processed_data_df['income'] == '40,000 - $49,999', 'income'] = 45000
processed_data_df.loc[processed_data_df['income'] == '50,000 - $59,999', 'income'] = 55000
processed_data_df.loc[processed_data_df['income'] == '$50,000 to $74,999', 'income'] = 62500
processed_data_df.loc[processed_data_df['income'] == '60,000 - $69,999', 'income'] = 65000
processed_data_df.loc[processed_data_df['income'] == '70,000 - $79,999', 'income'] = 75000
processed_data_df.loc[processed_data_df['income'] == '$75,000 to $99,999', 'income'] = 87500
processed_data_df.loc[processed_data_df['income'] == '80,000 - $89,999', 'income'] = 85000
processed_data_df.loc[processed_data_df['income'] == '90,000 - $99,999', 'income'] = 95000
processed_data_df.loc[processed_data_df['income'] == '100,000 and up', 'income'] = 100000
processed_data_df.loc[processed_data_df['income'] == '$100,000 to $149,999', 'income'] = 125000 
processed_data_df.loc[processed_data_df['income'] == '$150,000 or more', 'income'] = 150000

processed_data_df.loc[processed_data_df['age'] == '18-21', 'age'] = 24
processed_data_df.loc[processed_data_df['age'] == '18-24', 'age'] = 21
processed_data_df.loc[processed_data_df['age'] == '22-25', 'age'] = 23.5
processed_data_df.loc[processed_data_df['age'] == '25-34', 'age'] = 29.5
processed_data_df.loc[processed_data_df['age'] == '26-30', 'age'] = 28
processed_data_df.loc[processed_data_df['age'] == '31-40', 'age'] = 35.5
processed_data_df.loc[processed_data_df['age'] == '35-44', 'age'] = 39.5
processed_data_df.loc[processed_data_df['age'] == '41-50', 'age'] = 46.5
processed_data_df.loc[processed_data_df['age'] == '45-54', 'age'] = 48
processed_data_df.loc[processed_data_df['age'] == '51-60', 'age'] = 55.5
processed_data_df.loc[processed_data_df['age'] == '55-64', 'age'] = 58
processed_data_df.loc[processed_data_df['age'] == '61+', 'age'] = 61
processed_data_df.loc[processed_data_df['age'] == '65-74', 'age'] = 69.5
processed_data_df.loc[processed_data_df['age'] == '75 or older', 'age'] = 75
processed_data_df.head()


In [50]:
# To make sure all of your columns are stored as numbers, use the pd.to_numeric method like so.
processed_data_df = processed_data_df.apply(pd.to_numeric, errors='coerce')
# errors='coerce' will set things that can't be converted to numbers to NaN
# so you'll want to drop these like so.
processed_data_df = processed_data_df.dropna()
processed_data_df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category


In [51]:
# You can add a columns like so.
processed_data_df["inspection_score_Y_N"] = "N"
processed_data_df["risk_category_Y_N"] = "N"
processed_data_df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,inspection_score_Y_N,risk_category_Y_N


In [53]:
# And then you can customize these new coulmns using the same method as above. 
processed_data_df.loc[processed_data_df['inspection_score'] <= 75, 'inspection_score_Y_N'] = "Y"
processed_data_df.loc[processed_data_df['risk_category'] == 'Low Risk', 'risk_category_Y_N'] = "Y"
processed_data_df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,inspection_score_Y_N,risk_category_Y_N


In [56]:
# I'm now going to make a set of tables to be used in training some models
# The first set will be for linear regressions where the traget is numeric.
# Happiness
happy_lin_df = processed_data_df[[
                               'inspection_score', 
                               'risk_category', 
                               'violation_id'
                               ]].copy()
happy_lin_df.head()

,inspection_score,risk_category,violation_id


In [ ]:
# Satisfaction
sat_lin_df = processed_data_df[[
                               'satisfied', 
                               'age', 
                               'income'
                               ]].copy()
sat_lin_df.head()

In [ ]:
# The second set will be for classifiers where the target is a class.
# Happiness
happy_class_df = processed_data_df[[
                               'happy_Y_N', 
                               'age', 
                               'income'
                               ]].copy()
happy_class_df.head()

In [ ]:
# Satisfaction
sat_class_df = processed_data_df[[
                               'satisfied_Y_N', 
                               'age', 
                               'income'
                               ]].copy()
sat_class_df.head()

## Taining and Validation

Above I created four datasets worth exploring: 
- **`happy_lin_df`**. The data needed to access *happiness* along a continuous variable.
- **`sat_lin_df`**. The data needed to access *satisfaction* along a continuous variable.
- **`happy_class_df`**. The data needed to access *happiness* as a categorical variable.
- **`sat_class_df`**. The data needed to access *satisfaction* as a categorical variable.

Let's take them each in turn. 

## happy_lin_df



In [58]:
data = happy_lin_df

data = data[data["inspection_score"]<=70]

holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

ValueError: a must be greater than 0

In [59]:
sns.lmplot(x="inspection_score", y="risk_category", data=training, x_estimator=np.mean, order=1)

NameError: name 'training' is not defined

In [60]:
sns.lmplot(x="violation_id", y="inspection_score", data=training, x_estimator=np.mean, order=1)

NameError: name 'training' is not defined

In [61]:
model = ols("inspection_score ~ risk_category + violation_id", training).fit()
#model = ols("happy ~ age + income + np.power(age, 2) + np.power(income, 2)", training).fit()
model.summary()

NameError: name 'training' is not defined

In [ ]:
# Rerun with SciKitLearn because it's easy to check accuracy
features_train = training.drop("happy", axis=1).as_matrix(columns=None)
labels_train = training["happy"].as_matrix(columns=None)

features_test = holdout.drop("happy", axis=1).as_matrix(columns=None)
labels_test = holdout["happy"].as_matrix(columns=None)

lm = linear_model.LinearRegression()
clf = lm.fit(features_train, labels_train)
pred = clf.predict(features_test)
accuracy = metrics.r2_score(labels_test, pred)
print("R squared:",lm.score(features_train,labels_train))
print("Accuracy:",accuracy)

## sat_lin_df

In [ ]:
data = sat_lin_df

data = data[data["satisfied"]<=10]

holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

In [ ]:
sns.lmplot(x="age", y="satisfied", data=training, x_estimator=np.mean, order=1)

In [ ]:
sns.lmplot(x="income", y="satisfied", data=training, x_estimator=np.mean, order=1)

In [ ]:
model = ols("satisfied ~ age + income", training).fit()
model.summary()

In [ ]:
# Rerun with SciKitLearn because it's easy to check accuracy

features_train = training.drop("satisfied", axis=1).as_matrix(columns=None)
labels_train = training["satisfied"].as_matrix(columns=None)

features_test = holdout.drop("satisfied", axis=1).as_matrix(columns=None)
labels_test = holdout["satisfied"].as_matrix(columns=None)

lm = linear_model.LinearRegression()
clf = lm.fit(features_train, labels_train)
pred = clf.predict(features_test)
accuracy = metrics.r2_score(labels_test, pred)
print("R squared:",lm.score(features_train,labels_train))
print("Accuracy:",accuracy)

## happy_class_df

In [ ]:
data = happy_class_df
holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

# Define the target (y) and feature(s) (X)
features_train = training.drop("happy_Y_N", axis=1).as_matrix(columns=None)
labels_train = training["happy_Y_N"].as_matrix(columns=None)

features_test = holdout.drop("happy_Y_N", axis=1).as_matrix(columns=None)
labels_test = holdout["happy_Y_N"].as_matrix(columns=None)

# What percentage of the time is target Y?
print("Percentage of Ys: %s\n"%(len(data[data["happy_Y_N"]=="Y"])/len(data)))

#### initial visualization
feature_1_no = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_2_no = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_1_yes = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
feature_2_yes = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
plt.scatter(feature_1_yes, feature_2_yes, color = "g", label="Happy")
plt.scatter(feature_1_no, feature_2_no, color = "r", label="Unhappy")
plt.legend()
plt.xlabel("age")
plt.ylabel("income")
plt.show()


In [ ]:
# Logistic Regression
model = LogisticRegression(fit_intercept = False, C = 1e9)
clf = model.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Logistic Regression")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


# Test some spot
x_test = 70
y_test = 160000
print("")
print(clf.predict([[x_test,y_test]])[0])
print(clf.predict_proba([[x_test,y_test]])[0][1])
print("")

from sklearn import tree
clf = tree.DecisionTreeClassifier(min_samples_split=40)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("\nDecision Tree")
evaluate(pred, labels_test)
plot_bound("Y",holdout,1,2,0)



from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Random Forest")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


from sklearn.svm import SVC
clf = SVC(kernel="rbf",probability=True)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("SVM")
evaluate(pred, labels_test)  
#plot_bound("Y",holdout,1,2,0) # plot doesn't work with SVM

## sat_class_df

In [ ]:
data = sat_class_df
holdout = data.sample(frac=0.05)
training = data.loc[~data.index.isin(holdout.index)]

# Define the target (y) and feature(s) (X)
features_train = training.drop("satisfied_Y_N", axis=1).as_matrix(columns=None)
labels_train = training["satisfied_Y_N"].as_matrix(columns=None)

features_test = holdout.drop("satisfied_Y_N", axis=1).as_matrix(columns=None)
labels_test = holdout["satisfied_Y_N"].as_matrix(columns=None)

# What percentage of the time is target Y?
print("Percentage of Ys: %s\n"%(len(data[data["satisfied_Y_N"]=="Y"])/len(data)))

#### initial visualization
feature_1_no = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_2_no = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="N"]
feature_1_yes = [features_test[ii][0] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
feature_2_yes = [features_test[ii][1] for ii in range(0, len(features_test)) if labels_test[ii]=="Y"]
plt.scatter(feature_1_yes, feature_2_yes, color = "g", label="Happy")
plt.scatter(feature_1_no, feature_2_no, color = "r", label="Unhappy")
plt.legend()
plt.xlabel("age")
plt.ylabel("income")
plt.show()

In [ ]:
# Logistic Regression
model = LogisticRegression(fit_intercept = False, C = 1e9)
clf = model.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Logistic Regression")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


# Test some spot
x_test = 70
y_test = 160000
print("")
print(clf.predict([[x_test,y_test]])[0])
print(clf.predict_proba([[x_test,y_test]])[0][1])
print("")

from sklearn import tree
clf = tree.DecisionTreeClassifier(min_samples_split=40)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("\nDecision Tree")
evaluate(pred, labels_test)
plot_bound("Y",holdout,1,2,0)



from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("Random Forest")
evaluate(pred, labels_test)  
plot_bound("Y",holdout,1,2,0)


from sklearn.svm import SVC
clf = SVC(kernel="rbf",probability=True)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print("SVM")
evaluate(pred, labels_test)  
#plot_bound("Y",holdout,1,2,0) # plot doesn't work with SVM